In [ ]:
import requests
import pandas as pd
from pathlib import Path
import datetime

# ===================== CONFIGURACIÓN =====================

API_KEY = "bvEJzyh0h4J3czQcdJXHzasOSLbL9Bsr"   
COUNTRY = "MX"
YEAR = datetime.datetime.now().year
BASE_URL = "https://calendarific.com/api/v2/holidays"

BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "datasets"
DATA_DIR.mkdir(exist_ok=True)

OUT_FILE = DATA_DIR / "eventos_cdmx.csv"

# ===================== LLAMADA A LA API =====================

params = {
    "api_key": API_KEY,
    "country": COUNTRY,
    "year": YEAR,
}

print(f"Llamando a Calendarific para feriados de {COUNTRY} en {YEAR}...")
response = requests.get(BASE_URL, params=params)

if response.status_code != 200:
    print("Error al llamar a la API:", response.status_code, response.text)
    raise SystemExit(1)

data = response.json()

if "response" not in data or "holidays" not in data["response"]:
    print("Formato inesperado de respuesta:", data)
    raise SystemExit(1)

holidays = data["response"]["holidays"]

# ===================== PROCESAR Y NORMALIZAR DATOS =====================

rows = []
for holiday in holidays:
    name = holiday.get("name", "")
    desc = holiday.get("description", "")
    date_iso = holiday.get("date", {}).get("iso", "")
    types = holiday.get("type", [])
    type_str = ", ".join(types) if isinstance(types, list) else str(types)

    # Para tu pipeline anterior, queremos algo tipo: date,event,scope
    rows.append({
        "date": date_iso[:10],  # yyyy-mm-dd
        "event": name,
        "scope": type_str or "holiday"
    })

df = pd.DataFrame(rows)

# ===================== GUARDAR EN CSV =====================
df.to_csv(OUT_FILE, index=False, encoding="utf-8")

print(f"Se guardaron {len(df)} feriados en: {OUT_FILE}")
print(df.head())


Llamando a Calendarific para feriados de MX en 2025...
Se guardaron 45 feriados en: datasets/eventos_cdmx.csv
         date                  event             scope
0  2025-01-01         New Year's Day  National holiday
1  2025-01-06  Day of the Holy Kings        Observance
2  2025-02-02              Candlemas        Observance
3  2025-02-03       Constitution Day  National holiday
4  2025-02-05       Constitution Day        Observance
